In [41]:
import pandas as pd
import numpy as np

import time

import torch
from sentence_transformers import SentenceTransformer, util

## 1. Data Load

In [2]:
data = pd.read_csv("../data/total_data.csv")

In [3]:
display(data)

,title,description,url,data_type,source,ori_label,ori_source,label
0,충청남도 당진시_산업단지 입주기업 현황_20220304,"충청남도 당진시 산업단지 입주 기업 현황입니다.(연번, 단지명, 회사명, 업종명, ...",https://www.data.go.kr/data/15106898/fileData.do,텍스트,충청남도 당진시,산업·통상·중소기업,공공데이터포털,제조소비
1,전라남도 강진군_미곡처리장현황_20221006,"전라남도 강진군 미곡처리장 현황에 대한 데이터로소 시군, 명칭, 소재지, 관리부서,...",https://www.data.go.kr/data/15106993/fileData.do,텍스트,전라남도 강진군,농림,공공데이터포털,식품건강
2,경상남도 거창군_월별 코로나 확진자 현황_20221007,경상남도 거창군 월별 코로나19 관련 확진자 및 사망자 수 현황 데이터로 월별 확진...,https://www.data.go.kr/data/15098865/fileData.do,텍스트,경상남도 거창군,보건,공공데이터포털,보건의료
3,충청남도 부여군_1인당지방세부담액_20221006,상기 데이터는 연도별 주민 1인당 또는 세대 당 부담된 지방세액에 대한 정보로 조...,https://www.data.go.kr/data/15080007/fileData.do,텍스트,충청남도 부여군,재정·세제·금융,공공데이터포털,경제금융
4,전라남도 보성군_담배소매인지정현황_20221006,"전라남도 보성군 담배소매인지정현황에 관한 공공데이터로 업소명, 지번주소, 도로명주소...",https://www.data.go.kr/data/15035564/fileData.do,텍스트,전라남도 보성군,일반공공행정,공공데이터포털,행정법률
...,...,...,...,...,...,...,...,...
127046,서울시 구로구 식품첨가물제조업 인허가 정보,구로구의 식품을 제조 또는 가공할 때 기호가치를 향상시키거나 영양가치를 높일목적의 ...,https://data.seoul.go.kr/dataList/OA-18293/S/1...,"SHEET,OpenAPI",구로구 스마트도시정책관 빅데이터담당관,보건,서울데이터광장,보건의료
127047,서울시 송파구 국제회의시설업 인허가 정보,송파구의 컨벤션센터 등 대규모 관광 수요를 유발하는 국제회의를 개최할 수 있는 시설...,https://data.seoul.go.kr/dataList/OA-16990/S/1...,"SHEET,OpenAPI",송파구 스마트도시정책관 빅데이터담당관,문화/관광,서울데이터광장,문화관광
127048,서울시 중구 국제회의시설업 인허가 정보,중구의 컨벤션센터 등 대규모 관광 수요를 유발하는 국제회의를 개최할 수 있는 시설을...,https://data.seoul.go.kr/dataList/OA-16968/S/1...,"SHEET,OpenAPI",중구 스마트도시정책관 빅데이터담당관,문화/관광,서울데이터광장,문화관광
127049,서울시 강남구 국제회의시설업 인허가 정보,강남구의 컨벤션센터 등 대규모 관광 수요를 유발하는 국제회의를 개최할 수 있는 시설...,https://data.seoul.go.kr/dataList/OA-16989/S/1...,"SHEET,OpenAPI",강남구 스마트도시정책관 빅데이터담당관,문화/관광,서울데이터광장,문화관광


In [4]:
data.columns.values

array(['title', 'description', 'url', 'data_type', 'source', 'ori_label',
       'ori_source', 'label'], dtype=object)

In [5]:
des = data.description.values

In [6]:
[des[i] for i in range(20)]

['충청남도 당진시 산업단지 입주 기업 현황입니다.(연번, 단지명, 회사명, 업종명, 도로명주소, 지번주소, 위도, 경도, 전화번호, 데이터 기준일)',
 '전라남도 강진군 미곡처리장 현황에 대한 데이터로소 시군, 명칭, 소재지, 관리부서, 담당자 연락처 등에 대한 정보를 포함하고 있음',
 '경상남도 거창군 월별 코로나19 관련 확진자 및 사망자 수 현황 데이터로 월별 확진자 수와 사망자수 항목 데이터를 제공합니다.',
 '상기 데이터는 연도별 주민 1인당 또는 세대 당 부담된  지방세액에 대한 정보로 조세 부담 수준 확인 및 거시경제 조세승수 등의 산출 기초로 사용',
 '전라남도 보성군 담배소매인지정현황에 관한 공공데이터로 업소명, 지번주소, 도로명주소, 연락처 등의 정보를 제공하고 있습니다.',
 '공유재산 중 행정목적으로 사용하지 않거나 사용할 계획이 없는 일반재산에 대한 데이터 제공으로 공유재산의 효율적이 활용 가능',
 '전라남도 영광군의 장애인전용 주차구역 위반 과태료 부과건수 및 부과액자료로, 연도, 부과건수, 부과금액, 징수건수, 징수금액, 체납건수, 체납금액, 징수율, 데이터기준일자가 있습니다.',
 '서울시설공단에서 위탁운영중인 서울시립승화원의 월별 화장 통계자료로 대인, 소인, 행려, 개장, 사산, 무연고자로 구분하여 자료를 제공합니다.서울시립승화원 월별 화장 통계(2018~2020)',
 '대구광역시 남구 실내공기질 관리대상시설의 시설군, 시설명, 소재지도로명주소, 위도 등에 대한 데이터를 제공합니다.',
 '경상남도 거제시 미곡처리장 현황(미곡처리장 명칭, 소재지, 연락처, 위도, 경도, 기준일자)등에 대한 정보를 제공합니다.',
 '경기도 용인시에 등록된 연도별, 행정동별 전기차 등록현황에 대한 데이터입니다.데이터 기준: 2018년~2021년',
 '거창군 내 미곡처리장 현황(업체명, 대표전화번호, 지번주소, 도로명주소, 위도, 경도)에 대한 데이터 자료입니다.',
 '전라남도 목포시의 최근 3년간 장애인전용주차구역과태료부과현황에 

## 2. USE S-BERT

In [7]:
SBERT = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [8]:
ex = SBERT.encode(des[0])

In [9]:
ex[:10]

array([-0.6380399 , -0.49873194, -0.42911837,  0.7738545 ,  0.7459617 ,
        0.9840718 , -0.17374822, -0.5557989 ,  0.08503134, -0.6773501 ],
      dtype=float32)

In [10]:
len(ex)

768

In [11]:
ex2 = SBERT.encode(des[0])
ex2[:10]

array([-0.6380399 , -0.49873194, -0.42911837,  0.7738545 ,  0.7459617 ,
        0.9840718 , -0.17374822, -0.5557989 ,  0.08503134, -0.6773501 ],
      dtype=float32)

In [12]:
len(ex2)

768

In [13]:
target = data['description']

In [14]:
target

0         충청남도 당진시 산업단지 입주 기업 현황입니다.(연번, 단지명, 회사명, 업종명, ...
1         전라남도 강진군 미곡처리장 현황에 대한 데이터로소 시군, 명칭, 소재지, 관리부서,...
2         경상남도 거창군 월별 코로나19 관련 확진자 및 사망자 수 현황 데이터로 월별 확진...
3         상기 데이터는 연도별 주민 1인당 또는 세대 당 부담된  지방세액에 대한 정보로 조...
4         전라남도 보성군 담배소매인지정현황에 관한 공공데이터로 업소명, 지번주소, 도로명주소...
                                ...                        
127046    구로구의 식품을 제조 또는 가공할 때 기호가치를 향상시키거나 영양가치를 높일목적의 ...
127047    송파구의 컨벤션센터 등 대규모 관광 수요를 유발하는 국제회의를 개최할 수 있는 시설...
127048    중구의 컨벤션센터 등 대규모 관광 수요를 유발하는 국제회의를 개최할 수 있는 시설을...
127049    강남구의 컨벤션센터 등 대규모 관광 수요를 유발하는 국제회의를 개최할 수 있는 시설...
127050    송파구의 숙박 및 휴게 등의 목적으로 이용하는 장소 또는 시설에서 비디오물 시청기자...
Name: description, Length: 127051, dtype: object

In [18]:
start = time.time()
target_encode = SBERT.encode(target)
end = time.time()

print(f"{end - start:.5f} sec")

125.13707 sec


In [21]:
des_emb = torch.tensor(target_encode)

In [22]:
torch.save(des_emb, '../data/des_embedding.pt')

## 3. 유사한 데이터 찾기

In [24]:
import random
from random import randrange

random.seed(20221026)
random_idx = randrange(len(data))

In [25]:
random_idx

48252

In [83]:
def print_data_info(random_idx):
    print("##########################################")
    print('data title : ' + data['title'][random_idx])
    print('data description : ' + data['description'][random_idx])
    print('data url : ' + data['url'][random_idx])
    print('data type : ' + data['data_type'][random_idx])
    print('data source : ' + data['source'][random_idx])
    print('data ori_label : ' + data['ori_label'][random_idx])
    print('data ori_source : ' + data['ori_source'][random_idx])
    print('data label : ' + data['label'][random_idx])
    print("##########################################")
    print("\n")

In [40]:
print_data_info(random_idx)

data title : 국가평생교육진흥원_학부모지원시스템_상담_답변관리
data description : 학부모 상담관련 CSV파일
data url : https://www.data.go.kr/data/15071846/fileData.do
data type : CSV
data source : 공공데이터포털
data ori_label : 교육과학
data ori_source : 빅데이터지도
data label : 교육과학


In [50]:
distance = util.cos_sim(target_encode[random_idx] ,des_emb)

In [51]:
distance

tensor([[0.1428, 0.2793, 0.1772,  ..., 0.2250, 0.1997, 0.1859]])

In [53]:
len(distance[0])

127051

In [64]:
sort_distance = distance[0].sort()

In [68]:
sort_distance

torch.return_types.sort(
values=tensor([-0.1579, -0.1579, -0.1188,  ...,  0.9099,  0.9099,  1.0000]),
indices=tensor([109962,   4521,  92048,  ...,  35433,  35369,  48252]))

In [76]:
sort_distance.values[-6:-1]

tensor([0.9099, 0.9099, 0.9099, 0.9099, 0.9099])

In [77]:
sort_distance.indices[-6:-1]

tensor([48266, 48262, 35411, 35433, 35369])

In [84]:
for i in sort_distance.indices[-6:-1].tolist():
    print_data_info(i)

##########################################
data title : 국가평생교육진흥원_학부모지원시스템_상담_요구사항코드
data description : 학부모 상담결과 관련 CSV
data url : https://www.data.go.kr/data/15071883/fileData.do
data type : CSV
data source : 공공데이터포털
data ori_label : 교육과학
data ori_source : 빅데이터지도
data label : 교육과학
##########################################


##########################################
data title : 국가평생교육진흥원_학부모지원시스템_상담_상담신청리스트
data description : 학부모 상담결과 관련 CSV
data url : https://www.data.go.kr/data/15071872/fileData.do
data type : CSV
data source : 공공데이터포털
data ori_label : 교육과학
data ori_source : 빅데이터지도
data label : 교육과학
##########################################


##########################################
data title : 국가평생교육진흥원_학부모지원시스템_상담_상담분류코드_20201027
data description : 학부모 상담결과 관련 CSV
data url : https://www.data.go.kr/data/15071854/fileData.do
data type : 텍스트
data source : 국가평생교육진흥원
data ori_label : 교육
data ori_source : 공공데이터포털
data label : 교육과학
##########################################


#####